In [30]:
import pandas as pd
import requests
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
import re
import nltk



In [31]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
df = pd.read_excel("cik_list.xlsx")
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [33]:
y = "https://www.sec.gov/"
links = [y + x for x in df["SECFNAME"]]

In [34]:
reports = []
for url in links:
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")
    reports.append(soup.get_text())

print(f"Toatal {len(reports)} reports saved")


Toatal 152 reports saved


In [35]:
with open("Stopwords_Generic.txt","r")as f:
    stop_words = f.read()

stop_words = stop_words.split("\n")
print(f"Total number of stop Words are {len(stop_words)}")


Total number of stop Words are 121


In [36]:
master_dic = pd.read_csv("Loughran_McDonald.csv")
master_dic.head()

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Syllables,Source
0,AARDVARK,1,354,1.550080e-08,1.422600e-08,3.815486e-06,99,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.313627e-10,8.653817e-12,9.241714e-09,1,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,3.940882e-10,1.169679e-10,5.290465e-08,7,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,29,1.269840e-09,6.654735e-10,1.595100e-07,28,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8570,3.752595e-07,3.809464e-07,3.529356e-05,1108,0,0,0,0,0,0,0,3,12of12inf


In [37]:
positive_dictionary = [x for x in master_dic[master_dic["Positive"] != 0]["Word"]]
negetive_dictionary = [x for x in master_dic[master_dic["Negative"] != 0]["Word"]]

print(f"Total positive words in dictionary are {len(positive_dictionary)}")
print(f"Total negetive words in dictionary are {len(negetive_dictionary)}")


Total positive words in dictionary are 354
Total negetive words in dictionary are 2355


In [38]:
uncertainity = pd.read_excel("uncertainty_dictionary.xlsx")
uncertainity_words = list(uncertainity["Word"])

constraining = pd.read_excel("constraining_dictionary.xlsx")
constraining_words = list(constraining["Word"])


In [39]:
def tokenize(text):
    text = re.sub(r'[^A-Za-z]', " ",text.upper())
    tokenized_words = word_tokenize(text)
    return tokenized_words

def remove_stopwords(words, stop_words):
    return [x for x in words if x not in stop_words]

def countfnc(store, words):
    score = 0
    for x in words:
        if(x in store):
            score = score+1
    return score

def sentiment(score):
    if(score < -0.5):
        return "Most Negative"
    elif(score >= -0.5 and score < 0):
        return "Negative"
    elif(score == 0):
        return "Neutral"
    elif(score > 0 and score < 0.5):
        return "Positive"
    else:
        return "Very Positive"

def polarity(positive_score, negetive_score):
    return (positive_score - negetive_score)/((positive_score + negetive_score)+ 0.000001)


def subjectivity(positive_score, negative_score,num_words):
    return (positive_score+negative_score)/(num_words+ 0.000001)

def syllable_morethan2(word):
    if(len(word)> 2 and (word[-2:]=="es" or word[-2:] == "ed")):
        return False

    count =0
    vowels = ["a",'e','i','o','u']
    for i in word:
        if(i.lower() in vowels):
            count = count +1

    if(count > 2):
        return True
    else:
        return False

def fog_index_cal(average_sentence_length, percentage_complexwords):
    return 0.4*(average_sentence_length + percentage_complexwords)


In [40]:
sections = ["Management's Discussion and Analysis",
            "Quantitative and Qualitative Disclosures about Market Risk\n",
            "Risk Factors\n"]
caps = [x.upper() for x in sections]
caps.extend(sections)



In [41]:
col = ["mda",'qqdmr','rf']
var = ["positive_score",
       'negetive_score',
       'polarity_score',
       'average_sentence_legth',
       'percentage_of_complex_words',
       'fog_index',
       'complex_word_count',
       'word_count',
       'uncertainity_score',
       'constraining_score',
       'positive_word_proportion',
       'negetive_word_proportion',
       "uncertainity_word_proportion",
       'constraining_word_proportion',
       'constraining_words_whole_report']

for c in col:
    for v in var[:1]:
        df[c+"_"+v] = 0.0

df[var[-1]] = 0.0


In [42]:
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,qqdmr_positive_score,rf_positive_score,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,0.0,0.0,0.0,0.0
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,0.0,0.0,0.0,0.0
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0.0,0.0,0.0,0.0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,0.0,0.0,0.0,0.0
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0.0,0.0,0.0,0.0


In [43]:
section_map = {i:j for i, j in zip(sections, col)}
s_map = {i.upper():j for i,j in zip(sections, col)}

section_map.update(s_map)

In [44]:
for i in range(len(reports)):
    text = re.sub("Item",'ITEM',reports[i])
    for j in caps:
        x = re.search("ITEM\s+[\d]\(*A-Za-z]*.*\s+\-*\s*"+j, text)

        if x:
            start,end = x.span()
            content = (text[start:]).split("ITEM")[1]
            if ("..."not in content) and ("..."not in content) and len(content) > 200:
                tokenized_words = tokenize(content)
                words = remove_stopwords(tokenized_words, stop_words)
                num_words = len(words)
                positive_score = countfnc(positive_dictionary, words)
                negetive_score = countfnc(negetive_dictionary, words)

                polarity_score = polarity(positive_score, negetive_score)
                
                subjectivity_score  = subjectivity(positive_score, negetive_score, num_words)

                sentences = sent_tokenize(content)
                num_sentences = len(sentences)
                average_sentence_length = num_words/num_sentences

                num_complexword = 0
                uncertainity_score= 0
                constraining_score = 0

                for word in words:
                    if(syllable_morethan2(word)):
                        num_complexword = num_complexword+1

                    if(word in uncertainity_words):
                        uncertainity_score = uncertainity_score+1

                    if(word in constraining_words):
                        constraining_score = constraining_score+1

                    percentage_complexwords = num_complexword/num_words

                    fog_index = fog_index_cal(average_sentence_length, percentage_complexwords)

                    positive_word_proportion = positive_score/num_words
                    negative_word_proportion = negetive_score/num_words
                    uncertainity_word_proportion = uncertainity_score/num_words
                    constraining_word_proportion = constraining_score/num_words



                    df.at[i,section_map[j]+'_positive_score'] = positive_score
                    df.at[i,section_map[j]+'_negative_score'] = negative_score
                    df.at[i,section_map[j]+'_polarity_score'] = polarity_score
                    df.at[i,section_map[j]+'_average_sentence_length'] = average_sentence_length
                    df.at[i,section_map[j]+'_percentage_of_complex_words'] = percentage_complexwords
                    df.at[i,section_map[j]+'_fog_index'] = fog_index
                    df.at[i,section_map[j]+'_complex_word_count'] = num_complexword
                    df.at[i,section_map[j]+'_word_count'] = num_words
                    df.at[i,section_map[j]+'_uncertainity_score'] = uncertainity_score
                    df.at[i,section_map[j]+'_constraining_score'] = constraining_score
                    df.at[i,section_map[j]+'_positive_word_proportion'] = positive_word_proportion
                    df.at[i,section_map[j]+'_negative_word_proportion'] = negative_word_proportion
                    df.at[i,section_map[j]+'_uncertainity_word_proportion'] = uncertainity_word_proportion
                    df.at[i,section_map[j]+'_constraining_word_proportion'] = constraining_word_proportion


        constraining_words_whole_reports = 0
        tokenized_report_words = tokenize(reports[i])
        report_words = remove_stopwords(tokenized_report_words, stop_words)
        for word in report_words:
            if word in constraining_words:
                constraining_words_whole_reports = 1+ constraining_words_whole_reports
        df.at[i, "constraining_words_whole_report"]= constraining_words_whole_reports

                
                                


In [45]:
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,qqdmr_positive_score,rf_positive_score,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,0.0,0.0,0.0,0.0
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,0.0,0.0,0.0,0.0
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0.0,0.0,0.0,0.0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,0.0,0.0,0.0,0.0
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0.0,0.0,0.0,0.0


In [47]:
df.to_excel("output.xlsx")